In [ ]:
!hostname

# Model Loading
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig
import transformers
import torch
torch.set_grad_enabled(False)

path="./llama_hf/13b"

tokenizer = LlamaTokenizer.from_pretrained(path, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

model = LlamaForCausalLM.from_pretrained(path, device_map="cuda", load_in_8bit=True)

In [ ]:
import torch

def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"Number of GPUs: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"\nGPU {i}:")
            gpu_props = torch.cuda.get_device_properties(i)
            print(f"Total Memory: {gpu_props.total_memory / 1024**3:.2f} GB")
            print(f"Memory Allocated: {torch.cuda.memory_allocated(i) / 1024**3:.2f} GB")
            print(f"Memory Reserved: {torch.cuda.memory_reserved(i) / 1024**3:.2f} GB")
    else:
        print("CUDA is not available.")

print_gpu_memory()

!nvidia-smi

In [ ]:
# Generate prompts

# from tools import task_generation
# importlib.reload(task_generation)

from tools.task_generation import gen_task

batch_size = 200
task=gen_task(prompt_num=batch_size, example_num=5, same_task=False, seed=0)
task_target=gen_task(prompt_num=batch_size, example_num=5, same_task=False, seed=1)
print(task_target.answers)

from copy import deepcopy

def reshape(response:str):
    response_re=deepcopy(response)
    while response_re!="" and (response_re[0]==' ' or response_re[0] == '\n'):
        response_re=response_re[1:]
    response_re=response_re.split('\n')[0]
    response_re=response_re.split(' ')[0]
    return response_re

In [ ]:
# Inference and calculate accuracy

import numpy as np
from tools.inference import forward, modified_forward
from tools.utils import show_outputs, extract_hidden

def match_count(responses:list, answers:list, to_chr=-1):
    responses_mod=[None for _ in range(len(responses))]
    answers_mod=[None for _ in range(len(answers))]
    
    for index in range(len(responses)):
        res_len=len(reshape(responses[index]))
        ans_len=len(reshape(answers[index]))
        if to_chr>0:
            mod_len=min(to_chr,res_len)
        else:
            mod_len=res_len

        if mod_len>0:
            responses_mod[index]=reshape(responses[index])[:min(mod_len,ans_len)]
            answers_mod[index]=reshape(answers[index])[:min(mod_len,ans_len)]
        else:
            responses_mod[index]=""
            answers_mod[index]=answers[index]

    count=(np.array(responses_mod)==np.array(answers_mod)).sum()
    total_num=len(responses)
    return count/total_num

record=[]
for lg_id in [0,1,2]:
    print(f"Language id: {lg_id}")
    prompts=task.few_shot[lg_id]
    outputs=forward(model, tokenizer, prompts, with_hiddens=True)

    prompts_icl = task_target.few_shot[lg_id]
    outputs_icl=forward(model, tokenizer, prompts_icl, gen_len=4)
    responses_icl=show_outputs(tokenizer, outputs_icl, without_prompt=prompts_icl)

    prompts_raw = task_target.zero_shot[lg_id]
    outputs_raw=forward(model, tokenizer, prompts_raw, gen_len=4)
    responses_raw=show_outputs(tokenizer, outputs_raw, without_prompt=prompts_raw)

    answers = task_target.answers

    acc_raw=match_count(responses_raw, answers)
    acc_icl=match_count(responses_icl, answers)
    print(f"Original  {acc_raw:.2f}")
    print(f"ICL       {acc_icl:.2f}")

    t_v_acc_list=[]
    for target_layer_id in range(10,16):
        hiddens_tv=extract_hidden(outputs, layer_id=target_layer_id)
        outputs_raw_tv=modified_forward(model, tokenizer, prompts_raw, target_layer_id, hiddens_tv, gen_len=4)
        responses_raw_tv=show_outputs(tokenizer, outputs_raw_tv, without_prompt=prompts_raw)
        t_v_acc=match_count(responses_raw_tv, answers)
        t_v_acc_list.append(t_v_acc)
        print(f"Patching layer {target_layer_id:02d}  {t_v_acc:.2f}")

    t_v_acc_max=max(t_v_acc_list)
    record.append([acc_raw, acc_icl, t_v_acc_max])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

models = ['fr2en', 'sp2en', 'it2en']
conditions = ['few shot', 'task vector', 'zero shot']
accuracy = np.array(record)

df = pd.DataFrame(accuracy, index=models, columns=conditions).reset_index().melt(id_vars='index')
df.columns = ['Model', 'Condition', 'Accuracy']

sns.set_style("darkgrid")
g = sns.catplot(
    data=df, kind="bar",
    x="Accuracy", y="Model", hue="Condition",
    errorbar='sd', palette="bright",
    alpha=0.8, height=6, aspect=1, width=0.5,
)

# Adjust the layout
g.set_xticklabels([0,0.2,0.4,0.6,0.8,1])
# g.despine(left=True)
g.set_axis_labels("Accuracy", "Task")
g.legend.set_title("")

plt.show()

In [ ]:
from tools.task_generation import gen_task

batch_size = 200
task_train=gen_task(prompt_num=batch_size, example_num=5, same_task=False, seed=2)
task=gen_task(prompt_num=batch_size, example_num=5, same_task=False, seed=0)
task_target=gen_task(prompt_num=batch_size, example_num=5, same_task=False, seed=1)
print(task_target.answers)

In [ ]:
# Classification: proof of concept

import numpy as np
from tools.inference import forward, modified_forward
from tools.utils import show_outputs, extract_hidden

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

source_layer_id=39
target_layer_id=13

hiddens_list=[]
hiddens_tv_list=[]
for lg_id in [0,1,2]:
    prompts=task_train.few_shot[lg_id]
    outputs=forward(model, tokenizer, prompts, with_hiddens=True)
    hiddens=extract_hidden(outputs, layer_id=source_layer_id)
    hiddens_list.append(hiddens)
    hiddens_tv=extract_hidden(outputs, layer_id=target_layer_id)
    hiddens_tv_list.append(hiddens_tv)

DATA_train=torch.cat(hiddens_list,dim=0)
data_train=DATA_train.cpu().numpy()

# Training
classifier = LogisticRegression(max_iter=200)
classifier.fit(data_train, labels)

# Testing
hiddens_list=[]
hiddens_tv_list=[]
for lg_id in [0,1,2]:
    prompts=task.few_shot[lg_id]
    outputs=forward(model, tokenizer, prompts, with_hiddens=True)
    hiddens=extract_hidden(outputs, layer_id=source_layer_id)
    hiddens_list.append(hiddens)
    hiddens_tv=extract_hidden(outputs, layer_id=target_layer_id)
    hiddens_tv_list.append(hiddens_tv)

DATA_test=torch.cat(hiddens_list,dim=0)
data_test=DATA_test.cpu().numpy()

y_pred = classifier.predict(data_test)
accuracy = accuracy_score(labels, y_pred)

print(f"Accuracy: {accuracy:.3f}")

In [ ]:
# Inference and extract hidden states

import numpy as np
from tools.inference import forward, modified_forward
from tools.utils import show_outputs, extract_hidden

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def match_count(responses:list, answers:list, to_chr=-1):
    responses_mod=[None for _ in range(len(responses))]
    answers_mod=[None for _ in range(len(answers))]
    
    for index in range(len(responses)):
        res_len=len(reshape(responses[index]))
        ans_len=len(reshape(answers[index]))
        if to_chr>0:
            mod_len=min(to_chr,res_len)
        else:
            mod_len=res_len

        if mod_len>0:
            responses_mod[index]=reshape(responses[index])[:min(mod_len,ans_len)]
            answers_mod[index]=reshape(answers[index])[:min(mod_len,ans_len)]
        else:
            responses_mod[index]=""
            answers_mod[index]=answers[index]

    # print(responses_mod[0:10])
    # print(answers_mod[0:10])

    count=(np.array(responses_mod)==np.array(answers_mod)).sum()
    total_num=len(responses)
    return count/total_num

source_layer_id=39
target_layer_id=13

record2=[]
layer_list=[39]

for source_layer_id in layer_list:
    print(f"Recover: {source_layer_id} -> {target_layer_id}")

    hiddens_list=[]
    hiddens_tv_list=[]
    for lg_id in [0,1,2]:
        prompts=task_train.few_shot[lg_id]
        outputs=forward(model, tokenizer, prompts, with_hiddens=True)
        hiddens=extract_hidden(outputs, layer_id=source_layer_id)
        hiddens_list.append(hiddens)
        hiddens_tv=extract_hidden(outputs, layer_id=target_layer_id)
        hiddens_tv_list.append(hiddens_tv)

    DATA_last_train=torch.cat(hiddens_list,dim=0)
    DATA_tv_train=torch.cat(hiddens_tv_list,dim=0)
    X_train=DATA_last_train.cpu().numpy()
    Y_train=DATA_tv_train.cpu().numpy()

    ln_reg = LinearRegression()
    ln_reg.fit(X_train, Y_train)

In [ ]:
for lg_id in [0,1,2]:
    prompts=task.few_shot[lg_id]
    outputs=forward(model, tokenizer, prompts, with_hiddens=True)
    hiddens=extract_hidden(outputs, layer_id=source_layer_id)

    tv_pred = ln_reg.predict(hiddens.cpu().numpy())
    hiddens_tv_pred=torch.tensor(tv_pred, dtype=torch.float16).to("cuda")

    prompts_test=task_target.zero_shot[lg_id]
    answers_test=task_target.answers

    outputs_raw_tv=modified_forward(model, tokenizer, prompts_test, target_layer_id, hiddens_tv_pred, gen_len=3)
    responses_raw_tv=show_outputs(tokenizer, outputs_raw_tv, without_prompt=prompts_test)
    acc=match_count(responses_raw_tv, answers_test)
    print(f"Languge id:{lg_id}  Accuracy: {acc:.2f}")

In [ ]:
print(record)

In [ ]:
record_add=[[0.05, 0.84, 0.675, 0.64], [0.05, 0.83, 0.705, 0.69], [0.05, 0.885, 0.765, 0.75]]

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

models = ['fr2en', 'sp2en', 'it2en']
conditions = ['zero shot', 'few shot', 'task vector', 'recovered\ntask vector']
accuracy = np.array(record_add)

df = pd.DataFrame(accuracy, index=models, columns=conditions).reset_index().melt(id_vars='index')
df.columns = ['Model', 'Condition', 'Accuracy']

sns.set_style("darkgrid")
g = sns.catplot(
    data=df, kind="bar",
    x="Accuracy", y="Model", hue="Condition",
    errorbar='sd', palette="bright", alpha=0.8, height=6, aspect=1, width=0.5
)

# Adjust the layout
g.set_xticklabels([0,0.2,0.4,0.6,0.8,1])
# g.despine(left=True)
g.set_axis_labels("Accuracy", "Task")
g.legend.set_title("")

plt.show()
# plt.close()

In [ ]:
# Generate prompts

# from tools import task_generation
# importlib.reload(task_generation)

from tools.task_generation import gen_task

batch_size = 200
task=gen_task(prompt_num=batch_size, example_num=5, same_task=True, seed=0)

source_layer_id=39
target_layer_id=13

hiddens_list=[]
hiddens_tv_list=[]
for lg_id in [0,1,2]:
    prompts=task.few_shot[lg_id]
    outputs=forward(model, tokenizer, prompts, with_hiddens=True)
    hiddens=extract_hidden(outputs, layer_id=source_layer_id)
    hiddens_list.append(hiddens)
    hiddens_tv=extract_hidden(outputs, layer_id=target_layer_id)
    hiddens_tv_list.append(hiddens_tv)

DATA=torch.cat(hiddens_list,dim=0)
data=DATA.cpu().numpy()

DATA_tv=torch.cat(hiddens_tv_list,dim=0)
data_tv=DATA_tv.cpu().numpy()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

pca = PCA(n_components=2)
pca_result = pca.fit_transform(data)

tsne = TSNE(n_components=2, perplexity=30, learning_rate=200)
tsne_result = tsne.fit_transform(data)

# Plotting the explained variance
plt.figure(figsize=(8, 6))

label_list=["fr2en","sp2en","it2en"]
class_size=DATA.size(1)
for index in range(3):
    plt.title('PCA')
    pca_result_class = pca_result[batch_size*index:batch_size*(index+1),:]
    plt.scatter(pca_result_class[:, 0], pca_result_class[:, 1], label=label_list[index],alpha=0.7)
    # plt.title('tSNE')
    # tsne_result_class = tsne_result[batch_size*index:batch_size*(index+1),:]
    # plt.scatter(tsne_result_class[:, 0], tsne_result_class[:, 1], label=label_list[index],alpha=0.7)

plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend(loc=1)
plt.show()